<a href="https://colab.research.google.com/github/sprerak48/Autonomous-tagging-using-Deep-Learning/blob/Prerak/AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print('Our pythn shared notebook we can code here')

Our pythn shared notebook we can code here


In [0]:
from google.colab import files
uploaded = files.upload()